# Proyecto Banco

## Importar librerias

In [ ]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder




## Lectura de datos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Mineria de datos proyecto/datos_Banco.csv')

In [ ]:
df

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T173509,C1010011,5/8/83,M,NEW DELHI,120180.54,9/8/16,11229,356.0
1,T33671,C1010011,5/8/83,M,NOIDA,32500.73,26/9/16,123813,4750.0
2,T363022,C1010012,28/7/94,M,MUMBAI,24204.49,14/8/16,204409,1499.0
3,T251648,C1010014,19/8/84,M,MUMBAI,161848.76,7/8/16,220305,250.0
4,T89544,C1010014,19/8/84,M,MUMBAI,38377.14,1/8/16,154451,1205.0
...,...,...,...,...,...,...,...,...,...
1048562,T226394,C9099836,24/12/90,M,BHIWANDI,133067.23,7/8/16,5122,691.0
1048563,T980444,C9099877,9/6/96,M,BANGALORE,96063.46,15/9/16,120255,222.0
1048564,T401232,C9099919,21/10/93,M,GUNTUR,5559.75,18/8/16,122533,126.0
1048565,T659168,C9099941,22/4/95,M,CHENNAI,35295.92,28/8/16,213722,50.0


# ETL

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048567 entries, 0 to 1048566
Data columns (total 9 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   TransactionID            1048567 non-null  object 
 1   CustomerID               1048567 non-null  object 
 2   CustomerDOB              1046189 non-null  object 
 3   CustGender               1047764 non-null  object 
 4   CustLocation             1048416 non-null  object 
 5   CustAccountBalance       1046198 non-null  float64
 6   TransactionDate          1048567 non-null  object 
 7   TransactionTime          1048567 non-null  int64  
 8   TransactionAmount (INR)  1048567 non-null  float64
dtypes: float64(2), int64(1), object(6)
memory usage: 72.0+ MB


In [ ]:
df.shape

(1048567, 9)

In [ ]:
# Contar la cantidad de valores nulos en cada columna
null_counts = df.isnull().sum()

In [ ]:
null_counts

TransactionID                 0
CustomerID                    0
CustomerDOB                2378
CustGender                  803
CustLocation                151
CustAccountBalance         2369
TransactionDate               0
TransactionTime               0
TransactionAmount (INR)       0
dtype: int64

In [ ]:
# Eliminar la columna CustAccountBalance
df = df.drop('CustAccountBalance', axis=1)

In [ ]:
# Eliminar todos los registros con valores nulos
df = df.dropna()

In [ ]:
# Contar la cantidad de valores nulos en cada columna
null_counts = df.isnull().sum()
null_counts

TransactionID              0
CustomerID                 0
CustomerDOB                0
CustGender                 0
CustLocation               0
TransactionDate            0
TransactionTime            0
TransactionAmount (INR)    0
dtype: int64

In [ ]:

# Convertir CustomerDOB a fecha
df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'], errors='coerce')

# Convertir TransactionDate a fecha
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], errors='coerce')

# Ajustar los años para los valores NaT (no válidos) si es necesario
df['CustomerDOB'] = df['CustomerDOB'].apply(lambda x: x if x.year <= 2016 else x - pd.DateOffset(years=100))
df['TransactionDate'] = df['TransactionDate'].apply(lambda x: x if x.year <= 2016 else x - pd.DateOffset(years=100))


<ipython-input-65-2a64612aecf6>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['CustomerDOB'] = pd.to_datetime(df['CustomerDOB'], errors='coerce')
<ipython-input-65-2a64612aecf6>:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], errors='coerce')


# Calculo de datos

In [ ]:
# Acomodar el DataFrame primero por CustomerID y luego por TransactionDate
df_sorted = df.sort_values(by=['CustomerID', 'TransactionDate'])

In [ ]:
df_sorted

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T173509,C1010011,1983-05-08,M,NEW DELHI,2016-09-08,11229,356.0
1,T33671,C1010011,1983-05-08,M,NOIDA,2016-09-26,123813,4750.0
2,T363022,C1010012,1994-07-28,M,MUMBAI,2016-08-14,204409,1499.0
4,T89544,C1010014,1984-08-19,M,MUMBAI,2016-01-08,154451,1205.0
3,T251648,C1010014,1984-08-19,M,MUMBAI,2016-07-08,220305,250.0
...,...,...,...,...,...,...,...,...
1048562,T226394,C9099836,1990-12-24,M,BHIWANDI,2016-07-08,5122,691.0
1048563,T980444,C9099877,1996-09-06,M,BANGALORE,2016-09-15,120255,222.0
1048564,T401232,C9099919,1993-10-21,M,GUNTUR,2016-08-18,122533,126.0
1048565,T659168,C9099941,1995-04-22,M,CHENNAI,2016-08-28,213722,50.0


In [ ]:
df_sorted


,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T173509,C1010011,1983-05-08,M,NEW DELHI,2016-09-08,11229,356.0
1,T33671,C1010011,1983-05-08,M,NOIDA,2016-09-26,123813,4750.0
2,T363022,C1010012,1994-07-28,M,MUMBAI,2016-08-14,204409,1499.0
4,T89544,C1010014,1984-08-19,M,MUMBAI,2016-01-08,154451,1205.0
3,T251648,C1010014,1984-08-19,M,MUMBAI,2016-07-08,220305,250.0
...,...,...,...,...,...,...,...,...
1048562,T226394,C9099836,1990-12-24,M,BHIWANDI,2016-07-08,5122,691.0
1048563,T980444,C9099877,1996-09-06,M,BANGALORE,2016-09-15,120255,222.0
1048564,T401232,C9099919,1993-10-21,M,GUNTUR,2016-08-18,122533,126.0
1048565,T659168,C9099941,1995-04-22,M,CHENNAI,2016-08-28,213722,50.0


In [ ]:
# Ordena el DataFrame por CustomerID y Transaction Date
df.sort_values(by=['CustomerID', 'TransactionDate'], inplace=True)

# Calcula la diferencia entre las fechas de transacciones
df['Date Difference'] = df.groupby('CustomerID')['TransactionDate'].diff()

# Encuentra el último día del último mes en la columna 'TransactionDate'
ultimo_dia_ultimo_mes = df['TransactionDate'].max().replace(day=1).to_period('M').to_timestamp('M') + pd.offsets.MonthEnd(0)

# Calcula la diferencia de fecha faltante utilizando el último día del último mes
df['Date Difference'].fillna(ultimo_dia_ultimo_mes - df['TransactionDate'], inplace=True)


In [ ]:
df

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,TransactionDate,TransactionTime,TransactionAmount (INR),Date Difference
0,T173509,C1010011,1983-05-08,M,NEW DELHI,2016-09-08,11229,356.0,114 days
1,T33671,C1010011,1983-05-08,M,NOIDA,2016-09-26,123813,4750.0,18 days
2,T363022,C1010012,1994-07-28,M,MUMBAI,2016-08-14,204409,1499.0,139 days
4,T89544,C1010014,1984-08-19,M,MUMBAI,2016-01-08,154451,1205.0,358 days
3,T251648,C1010014,1984-08-19,M,MUMBAI,2016-07-08,220305,250.0,182 days
...,...,...,...,...,...,...,...,...,...
1048562,T226394,C9099836,1990-12-24,M,BHIWANDI,2016-07-08,5122,691.0,176 days
1048563,T980444,C9099877,1996-09-06,M,BANGALORE,2016-09-15,120255,222.0,107 days
1048564,T401232,C9099919,1993-10-21,M,GUNTUR,2016-08-18,122533,126.0,135 days
1048565,T659168,C9099941,1995-04-22,M,CHENNAI,2016-08-28,213722,50.0,125 days


In [ ]:

# Supongamos que tienes un DataFrame llamado 'df' con las columnas mencionadas

# Agrupa por CustomerID y calcula el promedio de TransactionAmount (INR)
transaction_amount_avg = df.groupby('CustomerID')['TransactionAmount (INR)'].mean()

# Selecciona la última fecha de transacción para cada CustomerID
last_transaction_date = df.groupby('CustomerID')['TransactionDate'].max()

# Calcula el promedio de Date Difference
date_difference_avg = df.groupby('CustomerID')['Date Difference'].mean()

# Crea un nuevo DataFrame con las columnas requeridas
new_df = pd.DataFrame({
    'CustomerID': transaction_amount_avg.index,
    'TransactionAmount (INR)': transaction_amount_avg.values,
    'TransactionDate': last_transaction_date.values,
    'Date Difference': date_difference_avg.values

})

# Agrega las columnas CustomerDOB, CustGender y CustLocation según corresponda
# Encuentra el último día del último mes en la columna 'TransactionDate'
ultimo_dia_ultimo_mes = df['TransactionDate'].max().replace(day=1).to_period('M').to_timestamp('M') + pd.offsets.MonthEnd(0)

# Calcula la diferencia entre la última fecha de transacción y el último día del último mes
new_df['Difference_Last_Transaction_Last_Month'] = ultimo_dia_ultimo_mes - new_df['TransactionDate']

new_df['CustomerDOB'] = df['CustomerDOB']
new_df['CustGender'] = df['CustGender']
new_df['CustLocation'] = df['CustLocation']
# Ahora 'new_df' contiene la información solicitada


In [ ]:
new_df

,CustomerID,TransactionAmount (INR),TransactionDate,Date Difference,Difference_Last_Transaction_Last_Month,CustomerDOB,CustGender,CustLocation
0,C1010011,2553.0,2016-09-26,66 days,96 days,1983-05-08,M,NEW DELHI
1,C1010012,1499.0,2016-08-14,139 days,139 days,1983-05-08,M,NOIDA
2,C1010014,727.5,2016-07-08,270 days,176 days,1994-07-28,M,MUMBAI
3,C1010018,30.0,2016-09-15,107 days,107 days,1984-08-19,M,MUMBAI
4,C1010024,5000.0,2016-08-18,135 days,135 days,1984-08-19,M,MUMBAI
...,...,...,...,...,...,...,...,...
881018,C9099836,691.0,2016-07-08,176 days,176 days,2001-01-01,M,BARMER
881019,C9099877,222.0,2016-09-15,107 days,107 days,2001-01-01,M,KANDIVALI WEST MUMBAI
881020,C9099919,126.0,2016-08-18,135 days,135 days,2001-01-01,M,MEERUT
881021,C9099941,50.0,2016-08-28,125 days,125 days,1982-11-15,M,GURGAON


In [ ]:
# Calcula el promedio de la columna 'TransactionAmount (INR)'
promedio = new_df['TransactionAmount (INR)'].mean()

# Imprime el resultado
print("El promedio de TransactionAmount (INR) es:", promedio)



El promedio de TransactionAmount (INR) es: 1568.6747111110226


In [ ]:
# Extrae la parte numérica de la columna Date Difference
new_df['Date Difference (Days)'] = new_df['Date Difference'].dt.days
# Extrae la parte numérica de la columna Date Difference
new_df['Difference_Last_Transaction_Last_Month (Days)'] = new_df['Difference_Last_Transaction_Last_Month'].dt.days
# Eliminar las columnas 'Date Difference' y 'Difference_Last_Transaction_Last_Month' de new_df
new_df.drop(columns=['Date Difference', 'Difference_Last_Transaction_Last_Month'], inplace=True)


In [ ]:

# Último día de diciembre de 2016
ultimo_dia_2016 = pd.Timestamp('2016-12-31')

# Calcular la edad de cada cliente en el DataFrame new_df
new_df['Age'] = ultimo_dia_2016.year - new_df['CustomerDOB'].dt.year
new_df.loc[new_df['CustomerDOB'].dt.month > ultimo_dia_2016.month, 'Age'] -= 1


In [ ]:
new_df

,CustomerID,TransactionAmount (INR),TransactionDate,CustomerDOB,CustGender,CustLocation,Date Difference (Days),Difference_Last_Transaction_Last_Month (Days),Age
0,C1010011,2553.0,2016-09-26,1983-05-08,M,NEW DELHI,66,96,33.0
1,C1010012,1499.0,2016-08-14,1983-05-08,M,NOIDA,139,139,33.0
2,C1010014,727.5,2016-07-08,1994-07-28,M,MUMBAI,270,176,22.0
3,C1010018,30.0,2016-09-15,1984-08-19,M,MUMBAI,107,107,32.0
4,C1010024,5000.0,2016-08-18,1984-08-19,M,MUMBAI,135,135,32.0
...,...,...,...,...,...,...,...,...,...
881018,C9099836,691.0,2016-07-08,2001-01-01,M,BARMER,176,176,15.0
881019,C9099877,222.0,2016-09-15,2001-01-01,M,KANDIVALI WEST MUMBAI,107,107,15.0
881020,C9099919,126.0,2016-08-18,2001-01-01,M,MEERUT,135,135,15.0
881021,C9099941,50.0,2016-08-28,1982-11-15,M,GURGAON,125,125,34.0


# Definicion de caracteristicas

In [ ]:
# Eliminar todos los registros nulos de new_df
new_df.dropna(inplace=True)

In [ ]:
# Crear un codificador OneHotEncoder
encoder = OneHotEncoder()

# Codificar las variables 'CustGender' en el DataFrame new_df
encoded_features = encoder.fit_transform(new_df[['CustGender']])

# Obtener los nombres de las nuevas columnas generadas por OneHotEncoding
encoded_column_names = encoder.get_feature_names_out(['CustGender'])

# Convertir la matriz codificada en un DataFrame
encoded_df = pd.DataFrame(encoded_features.toarray(), columns=encoded_column_names)

# Concatenar el DataFrame codificado con el DataFrame original
new_df_encoded = pd.concat([new_df, encoded_df], axis=1)

# Eliminar las columnas originales 'CustGender'
new_df_encoded.drop(['CustGender'], axis=1, inplace=True)

# Mostrar el DataFrame resultante con las variables codificadas
new_df_encoded


,CustomerID,TransactionAmount (INR),TransactionDate,CustomerDOB,CustLocation,Date Difference (Days),Difference_Last_Transaction_Last_Month (Days),Age,CustGender_F,CustGender_M,CustGender_T
0,C1010011,2553.0,2016-09-26,1983-05-08,NEW DELHI,66.0,96.0,33.0,0.0,1.0,0.0
1,C1010012,1499.0,2016-08-14,1983-05-08,NOIDA,139.0,139.0,33.0,0.0,1.0,0.0
2,C1010014,727.5,2016-07-08,1994-07-28,MUMBAI,270.0,176.0,22.0,0.0,1.0,0.0
3,C1010018,30.0,2016-09-15,1984-08-19,MUMBAI,107.0,107.0,32.0,0.0,1.0,0.0
4,C1010024,5000.0,2016-08-18,1984-08-19,MUMBAI,135.0,135.0,32.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
876381,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,1.0,0.0,0.0
876439,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,0.0,1.0,0.0
877647,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,1.0,0.0,0.0
878139,NaN,NaN,NaT,NaT,NaN,NaN,NaN,NaN,0.0,1.0,0.0


In [ ]:
# Eliminar todos los registros nulos de new_df
new_df_encoded.dropna(inplace=True)
null_counts = new_df_encoded.isnull().sum()
null_counts

CustomerID                                       0
TransactionAmount (INR)                          0
TransactionDate                                  0
CustomerDOB                                      0
CustLocation                                     0
Date Difference (Days)                           0
Difference_Last_Transaction_Last_Month (Days)    0
Age                                              0
CustGender_F                                     0
CustGender_M                                     0
CustGender_T                                     0
dtype: int64

In [ ]:
# Seleccionar características relevantes
features = ['TransactionAmount (INR)', 'Date Difference (Days)', 'Age', 'CustGender_F',	'CustGender_M',	'CustGender_T'	]


In [ ]:
# Dividir los datos en conjunto de características (X) y variable objetivo (y)
X = new_df_encoded[features]  # Características
y = new_df_encoded['Difference_Last_Transaction_Last_Month (Days)']  # Variable objetivo



In [ ]:
# Contar y eliminar los valores nulos en las características (X) y la variable objetivo (y)
null_count_X = X.isnull().sum()
null_count_y = y.isnull().sum()

# Eliminar filas con valores nulos en las características (X) y la variable objetivo (y)
X = X.dropna()
y = y.dropna()

# Verificar que ahora tengan la misma cantidad de muestras
print("Número de muestras después de eliminar valores nulos en X:", len(X))
print("Número de muestras después de eliminar valores nulos en y:", len(y))

# Dividir los datos en conjuntos de entrenamiento y prueba (70% entrenamiento, 30% prueba)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


Número de muestras después de eliminar valores nulos en X: 875574
Número de muestras después de eliminar valores nulos en y: 875574


In [ ]:
# Filtrar el DataFrame para obtener las filas que están entre dos valores de edad
age_min = 51
age_max = 200
filtered_df = new_df_encoded[(new_df_encoded['Age'] >= age_min) & (new_df_encoded['Age'] <= age_max)]

# Contar el número de filas que cumplen con el criterio de búsqueda
cantidad = len(filtered_df)

# Calcular el porcentaje en base al total de filas en el DataFrame original
total_filas = len(new_df_encoded)
porcentaje = (cantidad / total_filas) * 100

print("Cantidad de personas entre {} y {} años: {}".format(age_min, age_max, cantidad))
print("Porcentaje que representa: {:.2f}%".format(porcentaje))

# Calcular el promedio de TransactionAmount (INR) solo para las filas filtradas
promedio_transaction_amount = filtered_df['TransactionAmount (INR)'].mean()

print("Promedio de TransactionAmount (INR) para personas entre {} y {} años: {:.2f}".format(age_min, age_max, promedio_transaction_amount))


Cantidad de personas entre 51 y 200 años: 33310
Porcentaje que representa: 3.80%
Promedio de TransactionAmount (INR) para personas entre 51 y 200 años: 1559.62


In [ ]:
# Inicializar y entrenar el modelo de regresión logística
logistic_model = LogisticRegression()
logistic_model.fit(X_train, y_train)

# Predecir las probabilidades de abandono en el conjunto de prueba
probabilities_abandono = logistic_model.predict_proba(X_test)[:, 1]

# Agregar las probabilidades como una columna al conjunto de prueba
X_test_with_probabilities = X_test.copy()
X_test_with_probabilities['Probabilidad_Abandono'] = probabilities_abandono


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
print("Score del modelo:", logistic_model.score(X_test, y_test))


Score del modelo: 0.022731685403524535


In [ ]:

new_df_encoded

,CustomerID,TransactionAmount (INR),TransactionDate,CustomerDOB,CustLocation,Date Difference (Days),Difference_Last_Transaction_Last_Month (Days),Age,CustGender_F,CustGender_M,CustGender_T
0,C1010011,2553.00,2016-09-26,1983-05-08,NEW DELHI,66.0,96.0,33.0,0.0,1.0,0.0
1,C1010012,1499.00,2016-08-14,1983-05-08,NOIDA,139.0,139.0,33.0,0.0,1.0,0.0
2,C1010014,727.50,2016-07-08,1994-07-28,MUMBAI,270.0,176.0,22.0,0.0,1.0,0.0
3,C1010018,30.00,2016-09-15,1984-08-19,MUMBAI,107.0,107.0,32.0,0.0,1.0,0.0
4,C1010024,5000.00,2016-08-18,1984-08-19,MUMBAI,135.0,135.0,32.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
878287,C9039638,318.35,2016-09-08,1981-09-24,BATHINDA,114.0,114.0,35.0,0.0,1.0,0.0
878288,C9039640,906.00,2016-08-30,1957-10-27,MUMBAI,123.0,123.0,59.0,0.0,1.0,0.0
878289,C9039641,1067.50,2016-10-09,1982-12-20,BANGALORE,255.0,83.0,34.0,0.0,1.0,0.0
878290,C9039644,700.00,2016-08-29,1983-03-03,NEW DELHI,124.0,124.0,33.0,0.0,1.0,0.0


In [ ]:
# Función para asignar puntajes basados en la probabilidad
def asignar_puntaje_probabilidad(probabilidad):
    if probabilidad < 0.25:
        return 10
    elif probabilidad < 0.5:
        return 20
    elif probabilidad < 0.75:
        return 30
    else:
        return 40

# Función para asignar puntajes basados en la cantidad promedio por transacción
def asignar_puntaje_cantidad_promedio(cantidad_promedio):
    if cantidad_promedio < 500:  # Ejemplo de umbrales arbitrarios
        return 5
    elif cantidad_promedio < 1000:
        return 10
    elif cantidad_promedio < 1500:
        return 15
    else:
        return 20

# Función para asignar puntajes basados en la edad
def asignar_puntaje_edad(edad):
    if edad < 30:
        return 5
    elif edad < 50:
        return 10
    else:
        return 15

# Función para asignar puntajes basados en la cantidad promedio de días entre transferencia
def asignar_puntaje_dias_promedio(dias_promedio):
    if dias_promedio < 10:  # Ejemplo de umbrales arbitrarios
        return 5
    elif dias_promedio < 20:
        return 10
    else:
        return 15

# Aplicar las funciones a los datos
X_test_with_probabilities['Puntaje_Probabilidad'] = X_test_with_probabilities['Probabilidad_Abandono'].apply(asignar_puntaje_probabilidad)
X_test_with_probabilities['Puntaje_Cantidad_Promedio'] = X_test_with_probabilities['TransactionAmount (INR)'].apply(asignar_puntaje_cantidad_promedio)
X_test_with_probabilities['Puntaje_Edad'] = X_test_with_probabilities['Age'].apply(asignar_puntaje_edad)
X_test_with_probabilities['Puntaje_Dias_Promedio'] = X_test_with_probabilities['Date Difference (Days)'].apply(asignar_puntaje_dias_promedio)

# Combinar los puntajes para obtener el puntaje final
X_test_with_probabilities['Puntaje_Final'] = X_test_with_probabilities['Puntaje_Probabilidad'] + X_test_with_probabilities['Puntaje_Cantidad_Promedio'] + X_test_with_probabilities['Puntaje_Edad'] + X_test_with_probabilities['Puntaje_Dias_Promedio']

# Imprimir el DataFrame con los puntajes finales
print(X_test_with_probabilities)


        TransactionAmount (INR)  Date Difference (Days)   Age  CustGender_F  \
166638                     50.0                   137.0  25.0           0.0   
863042                    315.0                   346.0  23.0           0.0   
494631                    100.0                   298.0  30.0           0.0   
838338                   3468.5                   218.0  35.0           0.0   
271514                   1554.0                    53.0  24.0           0.0   
...                         ...                     ...   ...           ...   
217895                    210.0                   104.0  27.0           0.0   
294335                    370.0                    83.0  25.0           0.0   
856588                    200.0                   326.0  22.0           0.0   
85498                     960.0                   126.0  31.0           0.0   
496748                  10992.0                   129.0  42.0           1.0   

        CustGender_M  CustGender_T  Probabilidad_Ab

In [ ]:
X_test_with_probabilities

,TransactionAmount (INR),Date Difference (Days),Age,CustGender_F,CustGender_M,CustGender_T,Probabilidad_Abandono,Puntaje_Probabilidad,Puntaje_Cantidad_Promedio,Puntaje_Edad,Puntaje_Dias_Promedio,Puntaje_Final
166638,50.0,137.0,25.0,0.0,1.0,0.0,0.010966,10,5,5,15,35
863042,315.0,346.0,23.0,0.0,1.0,0.0,0.003531,10,5,5,15,35
494631,100.0,298.0,30.0,0.0,1.0,0.0,0.004707,10,5,10,15,40
838338,3468.5,218.0,35.0,0.0,1.0,0.0,0.009606,10,20,10,15,55
271514,1554.0,53.0,24.0,0.0,1.0,0.0,0.017155,10,20,5,15,50
...,...,...,...,...,...,...,...,...,...,...,...,...
217895,210.0,104.0,27.0,0.0,1.0,0.0,0.012948,10,5,5,15,35
294335,370.0,83.0,25.0,0.0,1.0,0.0,0.014269,10,5,5,15,35
856588,200.0,326.0,22.0,0.0,1.0,0.0,0.003953,10,5,5,15,35
85498,960.0,126.0,31.0,0.0,1.0,0.0,0.012448,10,10,10,15,45


In [ ]:
def asignar_valor(row):
    if row['Puntaje_Final'] < 40:
        return 1
    elif 55 <= row['Puntaje_Final'] < 70:
        return 3
    elif row['Puntaje_Final'] >= 70:
        return 4
    else:
        return 2

# Aplica la función a cada fila del DataFrame
X_test_with_probabilities['Tarjeta'] = X_test_with_probabilities.apply(asignar_valor, axis=1)

# Ahora 'Tarjeta' contiene los valores según las condiciones que especificaste
print(X_test_with_probabilities[['Puntaje_Final', 'Tarjeta']])

        Puntaje_Final  Tarjeta
166638             35        1
863042             35        1
494631             40        2
838338             55        3
271514             50        2
...               ...      ...
217895             35        1
294335             35        1
856588             35        1
85498              45        2
496748             55        3

[262673 rows x 2 columns]


In [ ]:
# Crear un nuevo registro con datos ficticios
nuevo_registro = {
    'CustomerID': 'C1005',
    'TransactionAmount (INR)': 10000,
    'TransactionDate': '2024-05-20',
    'CustomerDOB': '1990-01-15',
    'CustLocation': 'CityA',
    'Date Difference (Days)': 50,
    'Difference_Last_Transaction_Last_Month (Days)': 10000,
    'Age': 70,
    'CustGender_F': 0,  # 1 si es femenino, 0 si no
    'CustGender_M': 1,  # 1 si es masculino, 0 si no
    'CustGender_T': 0   # 1 si es otro género, 0 si no
}


In [ ]:
# Convertir el nuevo registro en un DataFrame y luego agregarlo al DataFrame existente
nuevo_registro_df = pd.DataFrame(nuevo_registro, index=[0])  # Crear un DataFrame con un solo registro
nuevo_registro_df


,CustomerID,TransactionAmount (INR),TransactionDate,CustomerDOB,CustLocation,Date Difference (Days),Difference_Last_Transaction_Last_Month (Days),Age,CustGender_F,CustGender_M,CustGender_T
0,C1005,10000,2024-05-20,1990-01-15,CityA,50,10000,70,0,1,0


In [ ]:

# Definir las columnas del DataFrame
columnas = ['CustomerID', 'TransactionAmount (INR)', 'TransactionDate', 'CustomerDOB', 'CustLocation',
            'Date Difference (Days)', 'Difference_Last_Transaction_Last_Month (Days)', 'Age',
            'CustGender_F', 'CustGender_M', 'CustGender_T']

# Crear un nuevo DataFrame con el nuevo registro
nuevo_registro_df = pd.DataFrame([nuevo_registro], columns=columnas)

# Agregar el nuevo registro al DataFrame new_df_encoded
new_df_encoded = pd.concat([new_df_encoded, nuevo_registro_df], ignore_index=True)

# Verificar el nuevo registro
new_df_encoded.tail()


,CustomerID,TransactionAmount (INR),TransactionDate,CustomerDOB,CustLocation,Date Difference (Days),Difference_Last_Transaction_Last_Month (Days),Age,CustGender_F,CustGender_M,CustGender_T
875570,C9039640,906.0,2016-08-30 00:00:00,1957-10-27 00:00:00,MUMBAI,123.0,123.0,59.0,0.0,1.0,0.0
875571,C9039641,1067.5,2016-10-09 00:00:00,1982-12-20 00:00:00,BANGALORE,255.0,83.0,34.0,0.0,1.0,0.0
875572,C9039644,700.0,2016-08-29 00:00:00,1983-03-03 00:00:00,NEW DELHI,124.0,124.0,33.0,0.0,1.0,0.0
875573,C9039645,1230.0,2016-07-09 00:00:00,1987-08-05 00:00:00,AURANGABAD,175.0,175.0,29.0,0.0,1.0,0.0
875574,C1005,10000.0,2024-05-20,1990-01-15,CityA,50.0,10000.0,70.0,0.0,1.0,0.0


In [ ]:
# Calcular la probabilidad de abandono para el nuevo registro
nuevo_registro_proba = logistic_model.predict_proba(nuevo_registro_df[['TransactionAmount (INR)', 'Date Difference (Days)', 'Age', 'CustGender_F', 'CustGender_M', 'CustGender_T']])[0, 1]

# Calcular el puntaje de crédito para el nuevo registro
puntaje_probabilidad = asignar_puntaje_probabilidad(nuevo_registro_proba)
puntaje_cantidad_promedio = asignar_puntaje_cantidad_promedio(nuevo_registro_df['TransactionAmount (INR)'].iloc[0])
puntaje_edad = asignar_puntaje_edad(nuevo_registro_df['Age'].iloc[0])
puntaje_dias_promedio = asignar_puntaje_dias_promedio(nuevo_registro_df['Date Difference (Days)'].iloc[0])
puntaje_final = puntaje_probabilidad + puntaje_cantidad_promedio + puntaje_edad + puntaje_dias_promedio
tarjeta = asignar_valor({'Puntaje_Final': puntaje_final})  # Pasar el puntaje_final como un diccionario

# Imprimir la probabilidad de abandono y el puntaje de crédito para el nuevo registro
print("Probabilidad de abandono del nuevo registro:", nuevo_registro_proba)
print("Puntaje de crédito del nuevo registro:", puntaje_final)
print("Es candidato para la tarjeta:", tarjeta)


Probabilidad de abandono del nuevo registro: 0.024214092158689873
Puntaje de crédito del nuevo registro: 60
Es candidato para la tarjeta: 3


In [ ]:
import pickle

# Supongamos que tienes un modelo entrenado llamado 'modelo'
# Guarda el modelo en un archivo usando pickle
with open('modelo.pkl', 'wb') as archivo:
    pickle.dump(logistic_model, archivo)

# Descarga el archivo del modelo desde Colab
from google.colab import files
files.download('modelo.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>